In [ ]:
import requests
import pandas as pd
import os

# Lagrer API-nøkkel
API = "2a430097-ac9b-44e4-995e-a02afb9795b2"
SOURCE = "SN68860"

# Setter et intervall for start og slutt dato
start_dato = "2024-07-01"
slutt_dato = "2024-12-31"
reference_time = f"{start_dato}/{slutt_dato}"

# API-url fra Frost
BASE_URL = "https://frost.met.no/observations/v0.jsonld"

# Navngir kolonner for verdier som skal hentes fra APIen
elements = {
    "air_temperature": "Temperatur (°C)",
    "mean(air_pressure_at_sea_level P1D)": "Lufttrykk (hPa)",
    "sum(precipitation_amount P1D)": "Nedbør (mm)"
}

# Input for å bestemme om man skal hente ny data eller ikke
force_update = input("Vil du hente nye data fra Frost API? (ja/nei): ").strip().lower()

# Funksjon for å hente og lagre data
def fetch_data(element_id, label):
    filename = f"frost_{element_id.replace('(', '').replace(')', '').replace(' ', '_')}_jul-des_2024.csv"
    
    if os.path.exists(filename) and force_update != "ja":
        print("Bruker eksisterende fil:", filename)
        return

    print("Henter data for", label + "...")
    
    params = {
        "sources": SOURCE,
        "elements": element_id,
        "referencetime": reference_time
    }

    response = requests.get(BASE_URL, params=params, auth=(API, ""))
    
    if response.status_code == 200:
        raw_data = response.json()["data"]
        rows = []
        for entry in raw_data:
            date = entry["referenceTime"].split("T")[0]
            value = entry["observations"][0]["value"]
            rows.append((date, value))
        
        df = pd.DataFrame(rows, columns=["Dato", label])
        df.to_csv(filename, index=False, encoding="utf-8")
        print("Lagret til:", filename)
    
    else:
        print(f"Feil ved henting av {element_id}: {response.status_code}")
        print(response.text)

# Henter og lagre data for hvert element
for element_id, label in elements.items():
    fetch_data(element_id, label)
    

Henter data for Temperatur (°C)...
Lagret til: frost_air_temperature_jul-des_2024.csv
Henter data for Lufttrykk (hPa)...
Lagret til: frost_meanair_pressure_at_sea_level_P1D_jul-des_2024.csv
Henter data for Nedbør (mm)...
Lagret til: frost_sumprecipitation_amount_P1D_jul-des_2024.csv


In [3]:
# TEMPFIL 15MIN -> GJENOMSNITT DAG

import pandas as pd
# Leser inn filen med temperaturmålinger
df = pd.read_csv("frost_air_temperature_jul-des_2024.csv")

# Beregn gjennomsnittstemperatur per dag
daglig_gjennomsnitt = df.groupby("Dato")["Temperatur (°C)"].mean().reset_index()

# Lagre til ny CSV-fil
daglig_gjennomsnitt.to_csv("frost_air_temperature_daglig_gjennomsnitt.csv", index=False)

print("Daglig gjennomsnitt lagret i frost_air_temperature_daglig_gjennomsnitt.csv")


Daglig gjennomsnitt lagret i frost_air_temperature_daglig_gjennomsnitt.csv
